# 実験概要
## CMNによる特徴量抽出
**ケプストラム平均正規化** (Cepstrum Mean Normalization : CMN) を使用し、特徴量を抽出する。
切り出した波形に対して**離散フーリエ変換** (Discrete Fourier Transform : DFT) を行い、絶対値を取ることで**振幅スペクトル**を得る。
これに対して対数を取って**対数振幅スペクトル**に変換し、逆離散フーリエ変換 (Inverse Discrete Fourier Transform : IDFT) を行うことで、ケプストラム領域へと変換し、先頭50要素を切り出し、結合した100要素の配列を前処理として返す。
この前処理は
```
left, right, posture = pp.slicer("raw\\" + tester.**.**.value)
cepstrum = pp.cmn_denoise(left, right)
```
によって行われる。

## DNNによる寝姿勢分類
DNNによって前処理したデータから学習・分類を行う。

# モジュールのインポート

In [69]:
import numpy as np
import glob

# 自作モジュール
import datapath as dpath
import preprocess as pp

# DataLoaderの定義

In [70]:
# tester以外のrawを読み込む
for p in glob.glob(".\\raw\\LMH\\*\\", recursive=True):
    print(p)

In [71]:
e = dpath.LMH.H002.value.fl_center
print(e.value)

type, tester, mattress = dpath.getattributes(e, position=False)
print(f"type : {type},  tester : {tester},  mattress : {mattress}")
train_paths = eval(f"dpath.{type}.serch('{mattress}')")
print(train_paths)

raw\LMH\H002\fl_center
type : LMH,  tester : H002,  mattress : fl
[WindowsPath('raw/LMH/H002/fl_center'), WindowsPath('raw/LMH/H003/fl_center'), WindowsPath('raw/LMH/L001/fl_center'), WindowsPath('raw/LMH/L003/fl_center'), WindowsPath('raw/LMH/M001/fls_center'), WindowsPath('raw/LMH/M001/fld_center'), WindowsPath('raw/LMH/M001/fls_center'), WindowsPath('raw/LMH/M002/fls_center'), WindowsPath('raw/LMH/M002/fld_center'), WindowsPath('raw/LMH/M002/fls_center'), WindowsPath('raw/LMH/M003/fls_center'), WindowsPath('raw/LMH/M003/fld_center'), WindowsPath('raw/LMH/M003/fls_center'), WindowsPath('raw/LMH/M004/fls_center'), WindowsPath('raw/LMH/M004/fld_center'), WindowsPath('raw/LMH/M004/fls_center'), WindowsPath('raw/LMH/H002/fl_center'), WindowsPath('raw/LMH/H003/fl_center'), WindowsPath('raw/LMH/L001/fl_center'), WindowsPath('raw/LMH/L003/fl_center'), WindowsPath('raw/LMH/M001/fls_center'), WindowsPath('raw/LMH/M001/fld_center'), WindowsPath('raw/LMH/M001/fls_center'), WindowsPath('raw/LMH/

In [72]:
import torch
from typing import Tuple
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

# 学習用データセットを読み込むためのDataLoaderを定義
class train_datasets(Dataset):
    def __init__(self, identity, transforms=None):
        self.transform = transforms
        self.type, self.tester, self.mattress = dpath.getattributes(identity)
        print(self.type, self.tester, self.mattress)
        print("train")

        # テスト以外のrawを読み込む
        self.train_cepstrum = np.empty((0, 100))  # 100要素の配列
        self.train_posture = np.empty(0)  # 姿勢データ配列

        # mattressに該当するrawのpathを読み込む
        train_paths = eval(f"dpath.{self.type}.serch('{self.mattress}')")
        for p in train_paths:
            if identity.value == p:
                continue
            left, right, posture = pp.slicer(p)
            cepstrum = pp.cmn_denoise(left, right)
            for c in cepstrum:
                self.train_cepstrum = np.vstack((self.train_cepstrum, c)) if self.train_cepstrum.size else c
            self.train_posture = np.append(self.train_posture, posture) if self.train_posture.size else posture

    def __len__(self):
        return len(self.train_posture)

    def __getitem__(self, idx) -> Tuple[torch.tensor, torch.tensor]:
        cepstrum = torch.tensor(self.train_cepstrum[idx].reshape(1, -1), dtype=torch.float32)
        posture = torch.tensor(self.train_posture[idx]-1, dtype=torch.long)
        if self.transform:
            cepstrum = self.transform(cepstrum)
        return cepstrum, posture


# test用データセット
class test_datasets(Dataset):
    def __init__(self, identity, transform=None):
        self.transform = transform
        self.type, self.tester, self.mattress = dpath.getattributes(identity)
        print("test")

        # 前処理したrawを読み込む
        self.left, self.right, self.test_posture = pp.slicer(identity.value)
        self.test_cepstrum = pp.cmn_denoise(self.left, self.right)

    def __len__(self):
        return len(self.test_posture)

    def __getitem__(self, idx) -> Tuple[torch.tensor, torch.tensor]:
        cepstrum = torch.tensor(self.test_cepstrum[idx].reshape(1, -1), dtype=torch.float32)
        posture = torch.tensor(self.test_posture[idx]-1, dtype=torch.long)
        if self.transform:
            cepstrum = self.transform(cepstrum)
        return cepstrum, posture

# 学習＆検証

In [73]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


## モデル定義
1次元ResNetを定義する。nn.Conv1dを用いて残差ブロックを定義する。

In [88]:
import torch.nn as nn

# ボトルネック残差ブロックの定義（1D用）
class Bottleneck1D(nn.Module):
    expansion = 4  # ボトルネックの出力チャンネルは、最初の畳み込み層の出力チャンネルの4倍

    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(Bottleneck1D, self).__init__()
        # 1x1 畳み込み層（チャンネル数の縮小）
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size=1, stride=1, bias=False)
        self.bn1 = nn.BatchNorm1d(out_channels)
        # 3x3 畳み込み層
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm1d(out_channels)
        # 1x1 畳み込み層（チャンネル数の拡張）
        self.conv3 = nn.Conv1d(out_channels, out_channels * self.expansion, kernel_size=1, stride=1, bias=False)
        self.bn3 = nn.BatchNorm1d(out_channels * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

# ResNetの定義（1D用、ResNet-50）
class ResNet1D(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super(ResNet1D, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv1d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm1d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool1d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)

        self.avgpool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if stride != 1 or self.in_channels != out_channels * block.expansion:
            downsample = nn.Sequential(
                nn.Conv1d(self.in_channels, out_channels * block.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm1d(out_channels * block.expansion),
            )

        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.in_channels, out_channels))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

# ResNet-50の構造を定義
def resnet1d50(num_classes=4):
    return ResNet1D(Bottleneck1D, [3, 4, 6, 3], num_classes=num_classes)



ResNetを定義する

In [75]:
identity = dpath.LMH.H002.value.st_center

train = train_datasets(identity)
test = test_datasets(identity)
batch_size = 128

train_loader = DataLoader(
    train,
    batch_size=batch_size,
    shuffle=True,
    drop_last=True
)

test_loader = DataLoader(
    test,
    batch_size=batch_size,
    shuffle=True,
    drop_last=True
)

LMH H002 st
train
data[340 / 403]
data[186 / 208]
data[173 / 208]
data[192 / 324]
data[174 / 438]
data[220 / 481]
data[87 / 281]
data[340 / 403]
data[186 / 208]
data[173 / 208]
data[192 / 324]
data[174 / 438]
data[220 / 481]
data[87 / 281]
test
data[194 / 230]


## 学習

In [87]:
import torch.optim as optim

# モデルのインスタンス化
net = resnet1d50(num_classes=4).to(device)

# 誤差関数を交差エントロピーで計算
criterion = nn.CrossEntropyLoss()

# 最適化アルゴリズム
optimizer = optim.Adam(net.parameters(), lr = 1e-6)

# 学習
n_epoch = 100
for epoch in range(n_epoch):
    # 精度と損失の初期化
    train_acc, train_loss = 0, 0
    val_acc, val_loss = 0, 0
    n_train, n_test = 0, 0

    # 学習
    for train_input, train_label in train_loader:
        n_train += len(train_label)

        # 入力と正解ラベルをGPU上に移動
        input = train_input.to(device)
        label = train_label.to(device)
        # print(f'input : {input.shape}, label : {label.shape}')

        optimizer.zero_grad()
        output = net(input)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()

        predicted = torch.max(output, 1)[1]

        train_loss += loss.item()
        train_acc += (predicted == label).sum().item()

    # 検証
    for test_input, test_label in test_loader:
        n_test += len(test_label)

        test_input = test_input.to(device)
        test_label = test_label.to(device)

        test_output = net(test_input)
        test_loss = criterion(test_output, test_label)

        test_predicted = torch.max(test_output, 1)[1]

        val_loss += test_loss.item()
        val_acc += (test_predicted == test_label).sum().item()

    # 精度を確率に変換
    train_acc /= n_train
    val_acc /= n_test
    train_loss = train_loss * batch_size / n_train
    val_loss = val_loss * batch_size / n_test

    print(f"Epoch[{epoch+1}/{n_epoch}], | loss: {train_loss:.5f} | acc: {train_acc:.5f} | val_loss: {val_loss:.5f} | val_acc: {val_acc:.5f}")


Epoch[1/500], | loss: 1.81214 | acc: 0.28609 | val_loss: 1.82350 | val_acc: 0.27604
Epoch[2/500], | loss: 1.59358 | acc: 0.28906 | val_loss: 1.91654 | val_acc: 0.23958
Epoch[3/500], | loss: 1.54906 | acc: 0.28497 | val_loss: 1.81449 | val_acc: 0.22396
Epoch[4/500], | loss: 1.48984 | acc: 0.29315 | val_loss: 1.81823 | val_acc: 0.24479
Epoch[5/500], | loss: 1.49047 | acc: 0.28088 | val_loss: 1.85839 | val_acc: 0.20833
Epoch[6/500], | loss: 1.45752 | acc: 0.29874 | val_loss: 1.75058 | val_acc: 0.21354
Epoch[7/500], | loss: 1.44125 | acc: 0.30618 | val_loss: 1.66974 | val_acc: 0.22396
Epoch[8/500], | loss: 1.44245 | acc: 0.30915 | val_loss: 1.62230 | val_acc: 0.27604
Epoch[9/500], | loss: 1.44419 | acc: 0.29427 | val_loss: 1.74762 | val_acc: 0.20312
Epoch[10/500], | loss: 1.44011 | acc: 0.31250 | val_loss: 1.67760 | val_acc: 0.26562
Epoch[11/500], | loss: 1.43255 | acc: 0.31101 | val_loss: 1.72955 | val_acc: 0.20833
Epoch[12/500], | loss: 1.41792 | acc: 0.31362 | val_loss: 1.72188 | val_ac

KeyboardInterrupt: 